## Use GPT-4 to retrieve list of editors

In [5]:
from lib.openai import query_openai_api
from lib.google import run_google_search, download

journal_name = "Kritische Vierteljahresschrift für Gesetzgebung und Rechtswissenschaft"
instruction = f"Finde im folgenden Text die Herausgeber, Redaktion/Schriftleitung und Beirat der Zeitschrift '{journal_name}' und gebe sie im CSV-Format zurück mit den Spalten 'lastname', 'firstname', 'title', 'position', 'affiliation','role'. Die Spalte 'role' enthält entweder 'Herausgeber', 'Redaktion', 'Beirat', 'Schriftleitung' oder ist leer wenn nicht bestimmbar. Wenn keine passenden Informationen verfügbar sind, gebe nur den CSV-Header zurück. Setze alle Werte in den CSV-Spalten in Anführungszeichen."

google_query = f'{journal_name} (herausgeber | redaktion | beirat)'
urls = run_google_search(google_query, lang="de", exclude=['jstor.org'], num_results=1)
website_data = download(urls[0]) # get the content of the first website found
# compare performance of different GPT models
for model in ["gpt-4", "gpt-3.5-turbo-instruct"]:
    csv_data = query_openai_api(model, instruction, website_data, max_tokens=2000)
    print(f"\n\n{model}\n---------------\n")
    print(csv_data)



gpt-4
---------------

"lastname","firstname","title","position","affiliation","role"
"Albrecht","Peter-Alexis","Prof. Dr.","Herausgeber","Goethe-Universität Frankfurt am Main","Herausgeber"
"Braum","Stefan","Prof. Dr.","Herausgeber und Redaktionsmitglied","Universität Luxemburg","Herausgeber"
"Braum","Stefan","Prof. Dr.","Herausgeber und Redaktionsmitglied","Universität Luxemburg","Redaktion"
"Broemel","Roland","Prof. Dr.","Herausgeber und Redaktionsmitglied","Goethe-Universität Frankfurt am Main","Herausgeber"
"Broemel","Roland","Prof. Dr.","Herausgeber und Redaktionsmitglied","Goethe-Universität Frankfurt am Main","Redaktion"
"Duve","Thomas","Prof. Dr.","Herausgeber","Max-Planck-Institut für europäische Rechtsgeschichte, Frankfurt am Main","Herausgeber"
"Günther","Klaus","Prof. Dr.","Herausgeber","Goethe-Universität Frankfurt am Main","Herausgeber"
"Jaeger","Marc","","Herausgeber","Präsident des Gerichts der Europäischen Union (2007-2019), Luxembourg","Herausgeber"
"Lamanda","Vinc

Clearly, GPT-3.5 is not up to the task. Now arun the combined google / gpt-4 pipeline on the whole dataset

In [2]:
import io
import os
import pandas as pd
from lib.openai import query_openai_api
from lib.google import run_google_search, download
from tqdm.notebook import tqdm
from datetime import datetime
from time import sleep

# Load or initialize journals data
if os.path.isfile("data/juristische-zeitschriften-with-editor-count.csv"):
    df_journals = pd.read_csv("data/juristische-zeitschriften-with-editor-count.csv")
else:
    df_journals = pd.read_csv("data/juristische-zeitschriften-lobid-berichtigt.csv")
    df_journals['editor_count'] = None

# Load or initialize editors data
if os.path.isfile("data/editors.csv"):
    df_editors = pd.read_csv("data/editors.csv", encoding='utf-8', index_col=False)
else:
    df_editors = pd.DataFrame(columns=['journal_abbr', 'lastname', 'firstname', 'title', 'position', 'affiliation', 'role', 'website','retrieved_on'])

try:
    # Iterate over each journal
    for _, row in tqdm(df_journals.iterrows(), desc="Processing journals...", total=len(df_journals)):
        journal_name = row['journal']
        journal_abbr = row['abbreviation'].split(',')[0].strip()
        
        # Skip if already processed
        if df_editors['journal_abbr'].str.contains(journal_abbr).any() or \
            (df_journals[df_journals['abbreviation'] == journal_abbr]['editor_count'].notna()).all():
            num_editors = (df_editors['journal_abbr'] == journal_abbr).sum()
            df_journals.loc[df_journals['abbreviation'] == journal_abbr, 'editor_count'] = num_editors
            print(f"Already retrieved {num_editors} editors for '{journal_abbr}'." )
            continue
    
        # Run google query
        google_query = f'{journal_name} intext:herausgeber|herausgegeben|redakt|schriftleit|beirat'
        exclude=[r'jstor\.org']
        urls = run_google_search(google_query, lang="de", exclude=exclude, verbose=True)
        
        # ask GPT-4 to identify the editors in the found websites
        for website_url in urls:

            website_data = download(website_url, verbose=True)
            
            if website_data is None or website_data =="":
                print("Could not retrieve website content.")
                sleep(3) # to avoid google blocking
                continue
            
            print(f'Sending content to GPT-4 for analysis...')

            instruction = f"Finde im folgenden Text die Herausgeber, Redaktion/Schriftleitung und Beirat der Zeitschrift '{journal_name}' und gebe sie im CSV-Format zurück mit den Spalten 'lastname', 'firstname', 'title', 'position', 'affiliation','role'. Die Spalte 'role' enthält entweder 'Herausgeber', 'Redaktion', 'Beirat', 'Schriftleitung' oder ist leer wenn nicht bestimmbar. Wenn keine passenden Informationen verfügbar sind, gebe nur den CSV-Header zurück. Setze alle Werte in den CSV-Spalten in Anführungszeichen."
            
            csv_data = query_openai_api("gpt-4", instruction, website_data, max_tokens=1800, verbose=True)
            data = io.StringIO(csv_data)
            try:
                df = pd.read_csv(data)
            except:
                raise RuntimeError(f"Error while parsing response from gpt-4:\n{csv_data}")

            # Save number of editors found
            num_editors = len(df)
            df_journals.loc[df_journals['abbreviation'] == journal_abbr, 'editor_count'] = num_editors
            print(f"Found {num_editors} editors.")

            # If some were found, add them to the list
            if num_editors > 0: 
                df['website'] = website_url
                df['journal_abbr'] = journal_abbr
                df['retrieved_on'] = datetime.now().strftime('%Y-%m-%d')
                df_editors = pd.concat([df_editors, df], ignore_index=True)
                
                # Save updated df_editors back to file
                df_editors.to_csv("data/editors.csv", encoding='utf-8', index=False)
                
                # don't look at the other websites found to save GPT-4 tokens 
                break

            
except KeyboardInterrupt:
    print("Loop interrupted by user. Saving current data.")


finally:
    # Save the updated df_journals to a new file
    df_journals.to_csv("data/juristische-zeitschriften-with-editor-count.csv", index=False)


Processing journals...:   0%|          | 0/610 [00:00<?, ?it/s]

Already retrieved 17 editors for 'AUR'.
Already retrieved 24 editors for 'AbfallR'.
Already retrieved 12 editors for 'AL'.
Already retrieved 0 editors for 'AJP'.
Already retrieved 0 editors for 'AktStR'.
Already retrieved 4 editors for 'ASR'.
Already retrieved 13 editors for 'AnwBl'.
Already retrieved 0 editors for 'AGS'.
Already retrieved 24 editors for 'ANA-ZAR'.
Already retrieved 11 editors for 'apf'.
Already retrieved 9 editors for 'APR'.
Already retrieved 23 editors for 'AuA'.
Already retrieved 4 editors for 'AuR'.
Already retrieved 1 editors for 'ArbR'.
Already retrieved 0 editors for 'AiB'.
Already retrieved 0 editors for 'AP'.
Already retrieved 9 editors for 'AöR'.
Already retrieved 6 editors for 'AVR'.
Already retrieved 3 editors for 'AcP'.
Already retrieved 14 editors for 'AfkKR'.
Already retrieved 14 editors for 'ArchKrim'.
Already retrieved 2 editors for 'UFITA'.
Already retrieved 10 editors for 'AfP'.
Already retrieved 3 editors for 'ARSP'.
Already retrieved 10 editors for